##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Lite Model Maker を使用した物体検知

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/lite/models/modify/model_maker/object_detection"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/lite/models/modify/model_maker/object_detection.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/lite/models/modify/model_maker/object_detection.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub で表示</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/lite/models/modify/model_maker/object_detection.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

この colab ノートブックでは、[TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/models/modify/model_maker) を使用して、モバイルデバイスで画像内のサラダを検出できる、カスタム物体検知モデルをトレーニングする方法について説明します。

Model Maker ライブラリは、転移学習によって、カスタムデータセットを使用した TensorFlow Lite モデルのトレーニングプロセスを簡素化します。TensorFlow Lite モデルと独自のカスタムデータセットを維持すると、必要なトレーニングデータの量が減り、トレーニング時間が短くなります。

公開されている *Salads* データセットを使用します。これは、[Open Images Dataset V4](https://storage.googleapis.com/openimages/web/index.html) から作成されたデータセットです。

データセットの各画像には、次のクラスのいずれかのラベルが付けられた物体が含まれています。

- パン
- チーズ
- サラダ
- シーフード
- トマト

データセットには、各物体の位置を示すバウンディングボックスと物体のラベルが含まれます。

次に、データセットの画像の例を示します。

<br>

<img src="https://cloud.google.com/vision/automl/object-detection/docs/images/quickstart-preparing_a_dataset.png" class="">


## 前提条件


### 必要なパッケージをインストールする

まず、[GitHub repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) の Model Maker パッケージや評価で使用する pycocotools ライブラリなどの必要なパッケージをインストールします。

In [ ]:
!sudo apt -y install libportaudio2
!pip install -q --use-deprecated=legacy-resolver tflite-model-maker
!pip install -q pycocotools
!pip install -q opencv-python-headless==4.1.2.30
!pip uninstall -y tensorflow && pip install -q tensorflow==2.8.0

必要なパッケージをインポートします。

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

### データセットを準備する

ここでは、AutoML [クイックスタート](https://cloud.google.com/vision/automl/object-detection/docs/edge-quickstart#preparing_a_dataset)と同じデータセットを使用します。

*Salads* データセットは `gs://cloud-ml-data/img/openimage/csv/salads_ml_use.csv` にあります。

トレーニング用に 175 画像、検証用に 25 画像、テスト用に 25 画像があります。データセットには、`Salad`、`Seafood`、`Tomato`、`Baked goods`、`Cheese` の 5 つのクラスがあります。

<br>

データセットは CSV 形式です。

```
TRAINING,gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg,Salad,0.0,0.0954,,,0.977,0.957,,
VALIDATION,gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg,Seafood,0.0154,0.1538,,,1.0,0.802,,
TEST,gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg,Tomato,0.0,0.655,,,0.231,0.839,,
```

- 各行は、大きい方の画像の局所にある物体に対応します。各物体は、テストデータ、トレーニングデータ、または検証データとして指定されています。このノートブックの後半では、この意味について説明します。
- ここに含まれる 3 行は、`gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg` にある**同じ画像内の 3 つの異なる物体**を示します。
- 各行には、`Salad`、`Seafood`、`Tomato` などの異なるラベルがあります。
- 各画像には、左上と右下の頂点を使用して、バウンディングボックスが指定されます。

次に、これらの 3 行の視覚化を示します。

<br>

<img src="https://cloud.google.com/vision/automl/object-detection/docs/images/quickstart-preparing_a_dataset.png" class="">

独自の CSV ファイルを準備する方法と、有効なデータセットを作成するための最低要件の詳細については、[Preparing your training data](https://cloud.google.com/vision/automl/object-detection/docs/prepare) ガイドを参照してください。

Google Cloud を初めて使用する場合は、`gs://` URL の意味がわからないかもしれません。これは [Google Cloud Storage](https://cloud.google.com/storage) (GCS) の URL です。GCS のファイルを公開するか、[クライアントを認証](https://cloud.google.com/storage/docs/authentication#libauth) する場合は、Model Maker でローカルファイルと同じようにこれらのファイルを読み取ることができます。

ただし、Model Maker を使用するために Google Cloud の画像を保持する必要はありません。CSV ファイルでローカルパスを使用すれば、Model Maker が動作します。

## クイックスタート

物体検出モデルをトレーニングするには、次の 6 つの手順があります。

**ステップ 1. 物体検出モデルアーキテクチャを選択する。**

このチュートリアルでは、EfficientDet-Lite0 モデルを使用します。EfficientDet-Lite[0-4] は、[EfficientDet](https://arxiv.org/abs/1911.09070) アーキテクチャから派生した、モバイル/IoT に対応した物体検出モデルのファミリーです。

次に、各 EfficientDet-Lite モデルのパフォーマンスの比較を示します。

モデルアーキテクチャ | サイズ (MB)* | レイテンシ ( ミリ秒)** | 平均適合率***
--- | --- | --- | ---
EfficientDet-Lite0 | 4.4 | 37 | 25.69%
EfficientDet-Lite1 | 5.8 | 49 | 30.55%
EfficientDet-Lite2 | 7.2 | 69 | 33.97%
EfficientDet-Lite3 | 11.4 | 116 | 37.70%
EfficientDet-Lite4 | 19.9 | 260 | 41.96%

<i> * 整数量子化モデルのサイズ。<br> ** CPU で 4 スレッドを使用した Pixel 4 で測定されたレイテンシ。<br> *** 平均適合率は COCO 2017 検証データセットでの mAP (平均適合率)。</i>


In [ ]:
spec = model_spec.get('efficientdet_lite0')

**ステップ 2. データセットを読み込む。**

Model Maker は CSV 形式で入力データを取り込みます。`object_detector.DataLoader.from_csv` メソッドを使用して、データセットを読み込み、トレーニング画像、検証画像、およびテスト画像に分割します。

- トレーニング画像: サラダの材料を認識するように物体検出モデルをトレーニングするために使用されます。
- 検証画像: トレーニングプロセス中にモデルが確認しなかった画像です。[過学習](https://en.wikipedia.org/wiki/Overfitting)を回避するためにトレーニングを停止すべきタイミングを判断するために使用されます。
- テスト画像: 最終的なモデルのパフォーマンスを評価するために使用されます。

直接 Google Cloud Storage から CSV ファイルを読み込めますが、Model Maker を使用するために、Google Cloud に画像を保持する必要はありません。コンピュータでローカル CSV ファイルを指定すれば、Model Maker が動作します。

In [ ]:
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('gs://cloud-ml-data/img/openimage/csv/salads_ml_use.csv')

**ステップ 3. training データを使用して TensorFlow モデルをトレーニングする。**

- EfficientDet-Lite0 モデルでは、既定で、`epochs = 50` を使用します。これは、トレーニングデータセットを 50 回繰り返すことを意味します。トレーニング中には検証精度を確認し、過学習を避けるために早い段階で停止できます。
- ここでは、`batch_size = 8` を設定します。トレーニングデータセットで 175 画像を学習するのに、21 ステップ必要です。
- ヘッドレイヤーだけをトレーニングして精度を改善するのではなく、`train_whole_model=True` を設定して、モデル全体を微調整します。トレードオフとして、モデルのトレーニングの時間が長くなる場合があります。

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=8, train_whole_model=True, validation_data=validation_data)

**ステップ 4. test データでモデルを評価する。**

トレーニングデータセットの画像を使用して物体検出モデルをトレーニングした後は、テストデータセットにある残りの 25 画像を使用して、初めて出現する新しいデータに対するモデルのパフォーマンスを評価します。

既定のバッチサイズは 64 です。つまり、テストデータセットにある 25 画像を処理するのに、1 ステップ必要です。

評価メトリックは [COCO](https://cocodataset.org/#detection-eval) と同じです。

In [ ]:
model.evaluate(test_data)

**ステップ 5. TensorFlow Lite モデルをエクスポートする。**

物体検出モデルを TensorFlow Lite 形式にエクスポートするには、量子化されたモデルのエクスポート先となるフォルダを指定します。既定のトレーニング後の量子化手法は、完全整数量子化です。

In [ ]:
model.export(export_dir='.')

**ステップ 6: TensorFlow Lite モデルを評価する。**

TFLite にエクスポートするときには、複数の要因がモデルの精度に影響する可能性があります。

- [量子化](https://www.tensorflow.org/lite/performance/model_optimization)は、精度が多少低下しますが、モデルサイズが 4 分の 1 に削減されます。
- 元の TensorFlow モデルでは、後処理で、クラス単位の [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) が使用されます。一方、TFLite モデルでは、大幅に高速ですが、精度の低いグローバル NMS が使用されます。Keras は最大 100 件の検出を出力しますが、tflite は最大 25 件の出力を検出します。

このため、エクスポートされた TFLite モデルを評価し、その精度を元の TensorFlow モデルと比較する必要があります。

In [ ]:
model.evaluate_tflite('model.tflite', test_data)

TensorFlow Lite モデルファイルは、Colab の左サイドバーを使ってダウンロードできます。`model.tflite` ファイルを右クリックして `Download` を選択すると、ローカルコンピュータにダウンロードされます。

このモデルは、[TensorFlow Lite Task Library](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview) の [ObjectDetector API](https://www.tensorflow.org/lite/inference_with_metadata/task_library/object_detector) を使って、Android または iOS アプリに統合することができます。

実際のアプリでモデルがどのように使用されているかについては、[TFLite テキスト分類サンプルアプリ](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/android)を参照してください。

*注意: Android Studio Model Binding は物体検出をサポートしていないため、TensorFlow Lite Task Library を使用してください。*

## (任意) 画像での TFLite モデルのテスト

インターネットの画像を使用して、トレーニング済みの TFLite モデルをテストできます。

- 次の `INPUT_IMAGE_URL` を任意の入力画像で置き換えます。
- `DETECTION_THRESHOLD` を調整し、モデルの感度を変更します。しきい値が低いほど、取得される物体数が多くなりますが、誤検出も増えます。一方、しきい値が高くなると、確実に検出された物体のみが取得されます。

現在、Python でモデルを実行するには、一部のボイラープレートコードが必要ですが、モデルをモバイルアプリに統合するには、数行のコードだけで統合できます。

In [ ]:
#@title Load the trained TFLite model and define some visualization functions

import cv2

from PIL import Image

model_path = 'model.tflite'

# Load the labels into a list
classes = ['???'] * model.model_spec.config.num_classes
label_map = model.model_spec.config.label_map
for label_id, label_name in label_map.as_dict().items():
  classes[label_id-1] = label_name

# Define a list of colors for visualization
COLORS = np.random.randint(0, 255, size=(len(classes), 3), dtype=np.uint8)

def preprocess_image(image_path, input_size):
  """Preprocess the input image to feed to the TFLite model"""
  img = tf.io.read_file(image_path)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.uint8)
  original_image = img
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  resized_img = tf.cast(resized_img, dtype=tf.uint8)
  return resized_img, original_image


def detect_objects(interpreter, image, threshold):
  """Returns a list of detection results, each a dictionary of object info."""

  signature_fn = interpreter.get_signature_runner()

  # Feed the input image to the model
  output = signature_fn(images=image)

  # Get all outputs from the model
  count = int(np.squeeze(output['output_0']))
  scores = np.squeeze(output['output_1'])
  classes = np.squeeze(output['output_2'])
  boxes = np.squeeze(output['output_3'])

  results = []
  for i in range(count):
    if scores[i] >= threshold:
      result = {
        'bounding_box': boxes[i],
        'class_id': classes[i],
        'score': scores[i]
      }
      results.append(result)
  return results


def run_odt_and_draw_results(image_path, interpreter, threshold=0.5):
  """Run object detection on the input image and draw the detection results"""
  # Load the input shape required by the model
  _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

  # Load the input image and preprocess it
  preprocessed_image, original_image = preprocess_image(
      image_path,
      (input_height, input_width)
    )

  # Run object detection on the input image
  results = detect_objects(interpreter, preprocessed_image, threshold=threshold)

  # Plot the detection results on the input image
  original_image_np = original_image.numpy().astype(np.uint8)
  for obj in results:
    # Convert the object bounding box from relative coordinates to absolute
    # coordinates based on the original image resolution
    ymin, xmin, ymax, xmax = obj['bounding_box']
    xmin = int(xmin * original_image_np.shape[1])
    xmax = int(xmax * original_image_np.shape[1])
    ymin = int(ymin * original_image_np.shape[0])
    ymax = int(ymax * original_image_np.shape[0])

    # Find the class index of the current object
    class_id = int(obj['class_id'])

    # Draw the bounding box and label on the image
    color = [int(c) for c in COLORS[class_id]]
    cv2.rectangle(original_image_np, (xmin, ymin), (xmax, ymax), color, 2)
    # Make adjustments to make the label visible for all objects
    y = ymin - 15 if ymin - 15 > 15 else ymin + 15
    label = "{}: {:.0f}%".format(classes[class_id], obj['score'] * 100)
    cv2.putText(original_image_np, label, (xmin, y),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

  # Return the final image
  original_uint8 = original_image_np.astype(np.uint8)
  return original_uint8

In [ ]:
#@title Run object detection and show the detection results

INPUT_IMAGE_URL = "https://storage.googleapis.com/cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg" #@param {type:"string"}
DETECTION_THRESHOLD = 0.3 #@param {type:"number"}

TEMP_FILE = '/tmp/image.png'

!wget -q -O $TEMP_FILE $INPUT_IMAGE_URL
im = Image.open(TEMP_FILE)
im.thumbnail((512, 512), Image.ANTIALIAS)
im.save(TEMP_FILE, 'PNG')

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Run inference and draw detection result on the local copy of the original file
detection_result_image = run_odt_and_draw_results(
    TEMP_FILE,
    interpreter,
    threshold=DETECTION_THRESHOLD
)

# Show the detection result
Image.fromarray(detection_result_image)

## (任意) Edge TPU 向けのコンパイル

量子化された EfficientDet Lite モデルがあるため、コンパイルして、[Coral EdgeTPU](https://coral.ai/) にデプロイできます。

**ステップ 1. EdgeTPU コンパイラのインストール**

In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler

**ステップ 2. Edge TPU 数の選択、コンパイル**

EdgeTPU には 8MB の SRAM が搭載され、モデルパラメータをキャッシュに格納するために使用されます ([詳細](https://coral.ai/docs/edgetpu/compiler/#parameter-data-caching))。つまり、8MB より大きいモデルの場合、モデルパラメータを転送するために、推論時間が長くなります。この問題を回避するための方法の 1 つは、[モデルパイプライン](https://coral.ai/docs/edgetpu/pipeline/)です。モデルが、専用の EdgeTPU を設けられるセグメントに分割されます。これにより、レイテンシが大幅に改善されます。

次の表は、使用する Edge TPU 数の参考値として使用できます。大きいモデルでは、中間テンソルがオンチップメモリに適合しないため、単一の TPU には対応しません。

モデルアーキテクチャ | 最低 TPU | 推奨 TPU
--- | --- | ---
EfficientDet-Lite0 | 1 | 1
EfficientDet-Lite1 | 1 | 1
EfficientDet-Lite2 | 1 | 2
EfficientDet-Lite3 | 2 | 2
EfficientDet-Lite4 | 2 | 3

In [ ]:
NUMBER_OF_TPUS =  1#@param {type:"number"}

!edgetpu_compiler model.tflite --num_segments=$NUMBER_OF_TPUS

**ステップ 3. モデルのダウンロード、実行**

モデルがコンパイルされたら、EdgeTPU で実行し、物体を検出できます。まず、Colab の左のサイドバーを使用して、ダウンロード済みの TensorFlow Lite モデルファイルをダウンロードします。`model_edgetpu.tflite` ファイルを右クリックして `Download` を選択すると、ローカルコンピュータにダウンロードされます。

任意の方法でモデルを実行できます。検出の例は次のとおりです。

- [pycoral 検出](https://github.com/google-coral/pycoral/blob/master/examples/detect_image.py)
- [基本 TFLite 検出](https://github.com/google-coral/tflite/tree/master/python/examples/detection)
- [動画検出の例](https://github.com/google-coral/examples-camera)
- [libcoral C++ API](https://github.com/google-coral/libcoral)


## 高度な使用

このセクションでは、モデルの調整やトレーニングハイパーパラメータの調整など、いくつかの高度なトピックを説明します。

### データセットの読み込み

#### 独自のデータの読み込み

このチュートリアルでは独自のデータセットをアップロードして進めることができます。データセットをアップロードするには、Colab の左サイドバーを使ってください。

<img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_object_detection.png" alt="Upload File" class="">

データセットをクラウドにアップロードしない場合は、[ガイド](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker)に従ってローカルでライブラリを実行することもできます。

#### 別のデータ形式のデータを読み込む

Model Maker ライブラリでは、`object_detector.DataLoader.from_pascal_voc` メソッドを使用して、[PASCAL VOC](https://towardsdatascience.com/coco-data-format-for-object-detection-a4c5eaf518c5#:~:text=Pascal%20VOC%20is%20an%20XML,for%20training%2C%20testing%20and%20validation) 形式のデータを読み込むこともできます。[makesense.ai](https://www.makesense.ai/) と [LabelImg](https://github.com/tzutalin/labelImg) は、画像に注釈を付け、PASCAL VOC データ形式で XML ファイルとして注釈を保存できるツールです。

```python
object_detector.DataLoader.from_pascal_voc(image_dir, annotations_dir, label_map={1: "person", 2: "notperson"})
```


### EfficientDet モデルハイパーパラメータのカスタマイズ

調整できるモデルおよびトレーニングパイプラインパラメータは次のとおりです。

- `model_dir`: モデルチェックポイントファイルを保存する場所。設定されていない場合、一時ディレクトリが使用されます。
- `steps_per_execution`: トレーニング実行単位のステップ数。
- `moving_average_decay`: 浮動小数点数。トレーニング済みパラメータの移動平均を管理するために使用する減衰。
- `var_freeze_expr`: 凍結される (トレーニング中に変更されない) 変数のプレフィックス名をマッピングする正規表現。具体的には、コードベースで `re.match(var_freeze_expr, variable_name)` を使用して、凍結する変数をマッピングします。
- `tflite_max_detections`: 整数。既定では 25。TFLite モデルの出力検出の最大数。
- `strategy`:  使用する分散ストラテジを指定する文字列。'tpu'、'gpus'、または ’None' を使用できます。'tpu' は TPUStrategy を使用することを意味します。'gpus' はマルチ gpus で MirroredStrategy を使用することを意味します。'None' の場合、TF の既定値と OneDeviceStrategy を使用します。
- `tpu`: トレーニングで使用する Cloud TPU。Cloud TPU を作成したときに使用した名前、または grpc://ip.address.of.tpu:8470 url を指定してください。
- `use_xla`: ストラテジが tpu ではない場合でも、XLA を使用してください。ストラテジが tpu の場合、必ず XLA を使用します。そうすると、このフラグが有効ではなくなります。
- `profile`: プロファイルモードを有効にします。
- `debug`: デバッグモードを有効にします。

調整可能な他のパラメータは、[hparams_config.py](https://github.com/google/automl/blob/df451765d467c5ed78bbdfd632810bc1014b123e/efficientdet/hparams_config.py#L170) を参照してください。

たとえば、変数を名前プレフィックス `efficientnet` で凍結する`var_freeze_expr='efficientnet'` を設定できます (既定値は `'(efficientnet|fpn_cells|resample_p6)'`)。これにより、モデルはトレーニングできない変数を凍結し、トレーニング中に同じ値を保持できます。

```python
spec = model_spec.get('efficientdet_lite0')
spec.config.var_freeze_expr = 'efficientnet'
```

### モデルアーキテクチャを変更する

モデルアーキテクチャを変更するには、`model_spec` を変更します。たとえば、`model_spec` を EfficientDet-Lite4 モデルに変更します。

```python
spec = model_spec.get('efficientdet_lite4')
```

### トレーニングハイパーパラメータの調整

`create` 関数は、Model Maker ライブラリがモデルの作成に使用するドライバー関数です。`model_spec` パラメータによってモデルの仕様が定義されています。現在サポートされているクラスは、`object_detector.EfficientDetSpec` です。`create` 関数の構成は、次の手順で行います。

1. `model_spec` に従って、物体検出のモデルを作成します。
2. モデルをトレーニングします。既定のエポックと既定のバッチサイズは、`model_spec` オブジェクトの`epochs` および `batch_size` 変数によって設定されます。モデルの精度に影響する `epochs` や `batch_size` などのトレーニングハイパーパラメータも微調整できます。次に例を示します。

- `epochs`: 整数。既定値は 50。エポック数が多いほど精度が改善されますが、過学習が発生する可能性があります。
- `batch_size`: 整数。既定値は 64。1 つのトレーニングステップに使用するサンプル数。
- `train_whole_model`: ブール値既定値は False。True の場合、モデル全体をトレーニングします。False の場合、`var_freeze_expr` と一致しないレイヤーのみをトレーニングします。

たとえば、トレーニングするエポックを減らし、ヘッドレイヤーのみをトレーニングできます。エポック数を増やすと、結果の質が向上します。

```python
model = object_detector.create(train_data, model_spec=spec, epochs=10, validation_data=validation_data)
```

### 別の形式へのエクスポート

エクスポート形式は次のいずれかを使用できます。

- `ExportFormat.TFLITE`
- `ExportFormat.LABEL`
- `ExportFormat.SAVED_MODEL`


既定では、オンデバイス ML アプリケーションで後から使用できるように、モデル[メタデータ](https://www.tensorflow.org/lite/models/convert/metadata)を含む TensorFlow Lite モデルファイルのみがエクスポートされます。ラベルファイルはメタデータに埋め込まれます。

多くのオンデバイス ML アプリケーションでは、モデルサイズが重要な要因です。そのため、モデルを量子化して小さくし、実行速度を高めることをお勧めします。EfficientDet-Lite モデルでは、既定で、完全な整数量子化によって、モデルが量子化されます。詳細については、[ポストトレーニング量子化](https://www.tensorflow.org/lite/performance/post_training_quantization)を参照してください。

```python
model.export(export_dir='.')
```

モデルに関連する他のファイルをエクスポートして、検査を強化することもできます。たとえば、次のように、保存されたモデルとラベルファイルをエクスポートできます。

```python
model.export(export_dir='.', export_format=[ExportFormat.SAVED_MODEL, ExportFormat.LABEL])
```

### TensorFlow Lite モデルでポストトレーニング量子化をカスタマイズする

[ポストトレーニング量子化](https://www.tensorflow.org/lite/performance/post_training_quantization)は、モデルサイズと推論レイテンシを縮小できる変換テクニックです。このテクニックでは、モデル精度にほとんど影響することなく、CPU とハードウェアアクセラレータの推論速度も改善することができます。したがって、モデルを改善するために広く使われています。

Model Maker ライブラリは、モデルをエクスポートする際に、デフォルトのポストトレーニング量子化手法を適用します。ポストトレーニング量子化をカスタマイズするのであれば、Model Maker は、[QuantizationConfig](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/config/QuantizationConfig) を使った複数のポストトレーニング量子化オプションもサポートしています。例として、float16 量子化を見てみましょう。まず、量子化構成を定義します。

```python
config = QuantizationConfig.for_float16()
```

次に、その構成で TensorFlow Lite モデルをエクスポートします。

```python
model.export(export_dir='.', tflite_filename='model_fp16.tflite', quantization_config=config)
```

# その他の資料

技術的な詳細については、[物体検出](https://www.tensorflow.org/lite/examples/object_detection/overview)の例をご覧ください。詳細については、以下をご覧ください。

- TensorFlow Lite Model Maker の[ガイド](https://www.tensorflow.org/lite/models/modify/model_maker)と [API リファレンス](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker)
- タスクライブラリ: デプロイ用の [ObjectDetector](https://www.tensorflow.org/lite/inference_with_metadata/task_library/object_detector)
- エンドツーエンドリファレンスアプリ: [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/android)、[iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/ios)、および [Raspberry PI](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/raspberry_pi)
